# Stock API
Using [Yahoo Finance API](https://python-yahoofinance.readthedocs.io/en/latest/api.html) to get stock information.

# Import Libraries

In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import math
from os import walk

import yfinance as yf

# Data

In [115]:
path = "Sample Data/"
filenames = next(walk(path), (None, None, []))[2]
print("Number of Stocks:", len(filenames))

Number of Stocks: 43


In [116]:
tickerSymbols = [file.split("_", 1)[0] for file in filenames]
tickerSymbols[0:5]

['TSLA', 'AAPL', 'JNJ', 'JPM', 'AMZN']

In [117]:
index = 0
tickerSymbol = tickerSymbols[index]
tickerSymbol

'TSLA'

## Fetch stock data from yahoo finance

In [122]:
start = "2020-09-08"
end = "2021-01-29"
stock_data = yf.download(tickerSymbol, start="2016-01-04", end="2021-04-30")

[*********************100%***********************]  1 of 1 completed


In [123]:
stock_data['Return'] = 100 * (stock_data['Close'].pct_change())